In [ ]:
# code/mbatch_i-o.ipynb

#### Split and merge by sys for mbatch

In [1]:
%whos

Interactive namespace is empty.


In [2]:
import pandas as pd
import glob

In [3]:
ls mbatch/bk_mbatch/data/

inputs/             mbatch_r4/          oc_splt_in/         r5_splt_in/
mbatch/             mbatch_r5/          one_cncr_lgg/       r6_splt_in/
mbatch_oc/          mbatch_r6/          r4_splt_in/         readme_mb_data.txt


In [4]:
mkdir mbatch/bk_mbatch/data/r7_splt_in

In [5]:
mkdir mbatch/bk_mbatch/data/mbatch_r7

In [6]:
ls mbatch/bk_mbatch/data/

inputs/             mbatch_r5/          one_cncr_lgg/       r7_splt_in/
mbatch/             mbatch_r6/          r4_splt_in/         readme_mb_data.txt
mbatch_oc/          mbatch_r7/          r5_splt_in/
mbatch_r4/          oc_splt_in/         r6_splt_in/


In [44]:
ls mbatch/bk_mbatch/data/r6_splt_in/

brain_CPTAC_transcriptomics_ref.tsv
brain_HCMI_transcriptomics_src.tsv
breast-lobular_CPTAC_transcriptomics_ref.tsv
breast-lobular_cell_line_transcriptomics_src.tsv
breast_CPTAC_transcriptomics_ref.tsv
breast_HCMI_transcriptomics_src.tsv
colon-adeno_CPTAC_transcriptomics_ref.tsv
colon-adeno_cell_line_transcriptomics_src.tsv
colon_CPTAC_transcriptomics_ref.tsv
colon_HCMI_transcriptomics_src.tsv
glioblastoma_CPTAC_transcriptomics_ref.tsv
glioblastoma_cell_line_transcriptomics_src.tsv
head-neck_CPTAC_transcriptomics_ref.tsv
head-neck_cell_line_transcriptomics_src.tsv
lung-ad_CPTAC_transcriptomics_ref.tsv
lung-ad_HCMI_transcriptomics_src.tsv
lung-adeno_CPTAC_transcriptomics_ref.tsv
lung-adeno_cell_line_transcriptomics_src.tsv
lung-sq_CPTAC_transcriptomics_ref.tsv
lung-sq_HCMI_transcriptomics_src.tsv
ovarian_CPTAC_transcriptomics_ref.tsv
ovarian_HCMI_transcriptomics_src.tsv
pancreatic-adeno_CPTAC_transcriptomics_ref.tsv
pancreatic-adeno_cell_line_transcriptomics_src.tsv
pancreatic_CPTAC_tra

In [8]:
ls ../data/r7/

AML_transcriptomics_cell-line+BeatAML.tsv
breast/
colon-adeno_transcriptomics_cell-line+CPTAC.tsv
colon-nos-adeno_transcriptomics_HCMI+CPTAC.tsv
glioblastoma_transcriptomics_cell-line+CPTAC.tsv
lung-adeno_transcriptomics_cell-line+CPTAC.tsv
pancreatic-adeno_transcriptomics_cell-line+CPTAC.tsv
pancreatic-nos-ductal-ad_transcriptomics_HCMI+CPTAC.tsv
renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv


In [ ]:
# Split files from ../data/r7/
    # write split files from this notebook to r7_splt_in/
    # command line calls on R script from docker container
    # send to mkdir mbatch/bk_mbatch/data/mbatch_r7/ from the R script

    # checking for negs on front end - AML failed
"""
     * Paths are tied to Docker loc *
"""

In [11]:
r7_pths = sorted(glob.glob('../data/r7/*.tsv'))

In [12]:
r7_pths

['../data/r7/AML_transcriptomics_cell-line+BeatAML.tsv',
 '../data/r7/colon-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../data/r7/colon-nos-adeno_transcriptomics_HCMI+CPTAC.tsv',
 '../data/r7/glioblastoma_transcriptomics_cell-line+CPTAC.tsv',
 '../data/r7/lung-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../data/r7/pancreatic-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../data/r7/pancreatic-nos-ductal-ad_transcriptomics_HCMI+CPTAC.tsv',
 '../data/r7/renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv']

In [ ]:
systems.split('+')[0]

##### NMF zeros

In [ ]:
# Duplicate all genes, so apply to columns -> 20K to 40K
P1: 4,6,-3,4,-1,4 becomes two rows
P1_pos: 4,6,0,4,0,4
P2_neg: 0,0,3,0,1,0

#### r7 production run

In [13]:
r = 'r7'
value_counts = []
for fl_pth in r7_pths:
    print(fl_pth.split('/')[-1])
    ab = pd.read_csv(
        fl_pth,
        sep = '\t', index_col = 0)
    print('min val ', ab.iloc[:, 2:].min().min())
    if ab.iloc[:, 2:].min().min() < 0:
        print('neg values')
        print(' ')
        continue # to next
    print('pos vals, splitting to disk', r)
    # print(' ')
    print(len(ab))
    # break
    system_labels = ab.System.unique()
    cancer = fl_pth.split('_')[0].split('/')[-1]
    modality = fl_pth.split('_')[1].split('/')[-1]
    systems = fl_pth.split('_')[-1].split('.')[0]
    print(systems)
    # break
    src_ref = 'src'
    sys = system_labels[0] # Automation note:
                            # cancer model system pos 0,
                            # human project pos 1 - done
    print(sys)
    a = ab[ab.System == sys].copy()
    a.drop(columns = ['System'], inplace = True)
    a.drop(columns = ['Cancer_type'], inplace = True)
    print(len(a))
    a = a.T
    # break
    a.to_csv(
              'mbatch/bk_mbatch/data/'+r+'_splt_in/'+
              cancer+'_'+sys+'_'+modality+'_'+src_ref+'.tsv',
              sep = '\t')

    src_ref = 'ref'
    sys = system_labels[1]
    print(sys)
    b = ab[ab.System == sys].copy()    # Label datum to file name
    b.drop(columns = ['System'], inplace = True)
    b.drop(columns = ['Cancer_type'], inplace = True)
    print(len(b))
    b = b.T
    b.to_csv(
              'mbatch/bk_mbatch/data/'+r+'_splt_in/'+
              cancer+'_'+sys+'_'+modality+'_'+src_ref+'.tsv',
              sep = '\t')
    
    print(' ')
    # break

AML_transcriptomics_cell-line+BeatAML.tsv
min val  -9.38333119672696
neg values
 
colon-adeno_transcriptomics_cell-line+CPTAC.tsv
min val  0.0
pos vals, spitting to disk r7
165
cell-line+CPTAC
cell-line
60
cptac
105
 
colon-nos-adeno_transcriptomics_HCMI+CPTAC.tsv
min val  0.0
pos vals, spitting to disk r7
151
HCMI+CPTAC
hcmi
46
cptac
105
 
glioblastoma_transcriptomics_cell-line+CPTAC.tsv
min val  0.0
pos vals, spitting to disk r7
152
cell-line+CPTAC
cell-line
52
cptac
100
 
lung-adeno_transcriptomics_cell-line+CPTAC.tsv
min val  0.0
pos vals, spitting to disk r7
188
cell-line+CPTAC
cell-line
77
cptac
111
 
pancreatic-adeno_transcriptomics_cell-line+CPTAC.tsv
min val  0.0
pos vals, spitting to disk r7
195
cell-line+CPTAC
cell-line
55
cptac
140
 
pancreatic-nos-ductal-ad_transcriptomics_HCMI+CPTAC.tsv
min val  0.0
pos vals, spitting to disk r7
181
HCMI+CPTAC
hcmi
41
cptac
140
 
renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv
min val  0.0
pos vals, spitting to disk r7
127
cell-line+

In [15]:
ls mbatch/bk_mbatch/data/r7_splt_in

colon-adeno_cell-line_transcriptomics_src.tsv
colon-adeno_cptac_transcriptomics_ref.tsv
colon-nos-adeno_cptac_transcriptomics_ref.tsv
colon-nos-adeno_hcmi_transcriptomics_src.tsv
glioblastoma_cell-line_transcriptomics_src.tsv
glioblastoma_cptac_transcriptomics_ref.tsv
lung-adeno_cell-line_transcriptomics_src.tsv
lung-adeno_cptac_transcriptomics_ref.tsv
pancreatic-adeno_cell-line_transcriptomics_src.tsv
pancreatic-adeno_cptac_transcriptomics_ref.tsv
pancreatic-nos-ductal-ad_cptac_transcriptomics_ref.tsv
pancreatic-nos-ductal-ad_hcmi_transcriptomics_src.tsv
renal-clear-cell_cell-line_transcriptomics_src.tsv
renal-clear-cell_cptac_transcriptomics_ref.tsv


In [ ]:
# ^ copy this to the R script for copy / paste 14 time for src / ref

#### Mbatch out to results/mbatch-out/rn

In [50]:
pwd

'/Users/karlberb/Documents/work/2024/bnch_mrk/mdl-sys-bnchmrk/code'

In [66]:
ls mbatch/bk_mbatch/data/mbatch_r6/

EBNPlus_TrainAndValidateReplicates_Structures/
correctedResults_src_v00.tsv
correctedResults_src_v01.tsv
correctedResults_src_v02.tsv
correctedResults_src_v03.tsv
correctedResults_src_v04.tsv
correctedResults_src_v05.tsv
correctedResults_src_v06.tsv
correctedResults_src_v07.tsv
correctedResults_src_v08.tsv
correctedResults_src_v09.tsv
correctedResults_src_v10.tsv
correctedResults_src_v11.tsv
correctedResults_src_v12.tsv
correctedResults_src_v13.tsv
input_ref_v00.tsv
input_ref_v01.tsv
input_ref_v02.tsv
input_ref_v03.tsv
input_ref_v04.tsv
input_ref_v05.tsv
input_ref_v06.tsv
input_ref_v07.tsv
input_ref_v08.tsv
input_ref_v09.tsv
input_ref_v10.tsv
input_ref_v11.tsv
input_ref_v12.tsv
input_ref_v13.tsv
input_src_v00.tsv
input_src_v01.tsv
input_src_v02.tsv
input_src_v03.tsv
input_src_v04.tsv
input_src_v05.tsv
input_src_v06.tsv
input_src_v07.tsv
input_src_v08.tsv
input_src_v09.tsv
input_src_v10.tsv
input_src_v11.tsv
input_src_v12.tsv
input_src_v13.tsv


In [45]:
ls ../results/mbatch-out/r6/r6-fls

ls: ../results/mbatch-out/r6/r6-fls: No such file or directory


In [47]:
r

'r6'

In [69]:
mb_n = sorted(glob.glob(
    'mbatch/bk_mbatch/data/mbatch_'+r+'/correctedResults_src_*.tsv'))
mb_n

In [71]:
mb_n

['mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v00.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v01.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v02.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v03.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v04.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v05.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v06.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v07.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v08.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v09.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v10.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v11.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v12.tsv',
 'mbatch/bk_mbatch/data/mbatch_r6/correctedResults_src_v13.tsv']

In [ ]:
# From R script 5
cancer_type = [
    'pancreatic',
    'ovarian',
    'lung-sq',
    'lung-ad',
    'breast',
    'brain',
    ]

In [ ]:
moda_type = [
    # 'pancreatic',
    # 'ovarian',
    # 'lung-sq',
    # 'lung-ad',
    # 'breast',
    # 'brain',
    ]

In [ ]:
sys_type = [
    'HCMI+CPTAC',
    'HCMI+CPTAC',
    'lung-sq',
    'lung-ad',
    'breast',
    'brain',
    ]

#### r6 name list

In [74]:
name_list = ['brain_transcriptomics_HCMI+CPTAC',
            'breast-lobular_transcriptomics_cell-line+CPTAC',
            'breast_transcriptomics_HCMI+CPTAC',
            'colon-adeno_transcriptomics_cell-line+CPTAC',
            'colon_transcriptomics_HCMI+CPTAC',
            'glioblastoma_transcriptomics_cell-line+CPTAC',
            'head-neck_transcriptomics_cell-line+CPTAC',
            'lung-ad_transcriptomics_HCMI+CPTAC',
            'lung-adeno_transcriptomics_cell-line+CPTAC',
            'lung-sq_transcriptomics_HCMI+CPTAC',
            'ovarian_transcriptomics_HCMI+CPTAC',
            'pancreatic-adeno_transcriptomics_cell-line+CPTAC',
            'pancreatic_transcriptomics_HCMI+CPTAC',
            'renal-clear-cell_transcriptomics_cell-line+CPTAC']

### r7 rebuild

In [17]:
mb_n = sorted(glob.glob(
    'mbatch/bk_mbatch/data/mbatch_'+r+'/correctedResults_src_*.tsv'))
mb_n

['mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v0colncell.tsv',
 'mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v1colnhcmi.tsv',
 'mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v2gliocell.tsv',
 'mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v3lungcell.tsv',
 'mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v4panccell.tsv',
 'mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v5panchcmi.tsv',
 'mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v6renlcell.tsv']

In [19]:
name_list = ['colon-adeno_transcriptomics_cell-line+CPTAC',
'colon-nos-adeno_transcriptomics_HCMI+CPTAC',
'glioblastoma_transcriptomics_cell-line+CPTAC',
'lung-adeno_transcriptomics_cell-line+CPTAC',
'pancreatic-adeno_transcriptomics_cell-line+CPTAC',
'pancreatic-nos-ductal-ad_transcriptomics_HCMI+CPTAC',
'renal-clear-cell_transcriptomics_cell-line+CPTAC',]

In [20]:
name_list

['colon-adeno_transcriptomics_cell-line+CPTAC',
 'colon-nos-adeno_transcriptomics_HCMI+CPTAC',
 'glioblastoma_transcriptomics_cell-line+CPTAC',
 'lung-adeno_transcriptomics_cell-line+CPTAC',
 'pancreatic-adeno_transcriptomics_cell-line+CPTAC',
 'pancreatic-nos-ductal-ad_transcriptomics_HCMI+CPTAC',
 'renal-clear-cell_transcriptomics_cell-line+CPTAC']

In [16]:
r

'r7'

In [21]:
# r7 rebuild
for i, pth in enumerate(mb_n):
    print(pth)
    dot_T = pd.read_csv(
        pth,
        sep = '\t', index_col = 0)

    # strctrd_dot_T
    df = dot_T.T

    df.insert(0, 'Cancer_type', name_list[i].split('_')[0])

    src_df = df[df.index.str.endswith(".src")]
    ref_df = df[df.index.str.endswith(".ref")]
    # break
    src_df.insert(0, 'System', name_list[i].split('_')[2].split('+')[0])

    ref_df.insert(0, 'System', name_list[i].split('_')[2].split('+')[1])

    mb = pd.concat([src_df, ref_df], axis = 0)
    # break
    mb.to_csv(
        '../results/'+r+'/mbatch/'+name_list[i]+'.tsv',
        sep = '\t')
    # break

mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v0colncell.tsv
mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v1colnhcmi.tsv
mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v2gliocell.tsv
mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v3lungcell.tsv
mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v4panccell.tsv
mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v5panchcmi.tsv
mbatch/bk_mbatch/data/mbatch_r7/correctedResults_src_v6renlcell.tsv


In [ ]:
mb

In [ ]:
# end v7
    # show proteomics correction on poster (bar plot?) with aml?
    # * remove aml from the ASW calc on the front end

In [ ]:
lst_file_out = '../results/mbatch/one_cncr/files/'+cancer+'_'+modality+'_'+systems+'.tsv'

In [ ]:
chk = pd.read_csv(
    lst_file_out,
        sep = '\t', index_col = 0)

In [ ]:
chk

##### devel notes on one cancer return to strctrd frmt:

In [ ]:
for indx in strctrd.index:
    print(indx) # observation: mbatch output is sorted alphanum

In [ ]:
ab = glob.glob('../strctrd/one_cncr/'+cancer_type+'*')

#### Two-cancer version

##### read and check

In [ ]:
ab = pd.read_csv( # changed to strctrd/one_cncr above
    '../output/dbl-cncr/pancreatic-adeno+lung-adeno_transcriptomics_cell-line+CPTAC.tsv',
        sep = '\t', index_col = 0)

In [ ]:
# mbatch takes non-negative numbers
ab.iloc[:, 2:].min().min()

In [ ]:
ab.System.value_counts()

In [ ]:
ab.Cancer_type.value_counts()

In [ ]:
cancers = 'pancreatic-adeno+lung-adeno'

In [ ]:
modality = 'transcriptomics'

##### Split template, using dep output/ paths

In [ ]:
src_ref = 'src'
sys = 'cell_line'
cl = ab[ab.System == 'cell_line'].copy()
cl.drop(columns = ['System'], inplace = True)
cl.drop(columns = ['Cancer_type'], inplace = True)
cl = cl.T
cl.to_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
    sep = '\t')

In [ ]:
cl_transposed_read_test = pd.read_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
        sep = '\t',
        index_col = 0)

In [ ]:
cl_transposed_read_test

In [ ]:
src_ref = 'ref'
sys = 'cptac'
cp = ab[ab.System == 'CPTAC'].copy()
cp.drop(columns = ['System'], inplace = True)
cp.drop(columns = ['Cancer_type'], inplace = True)
cp = cp.T
cp.to_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
    sep = '\t')

In [ ]:
cp_transposed_read_test = pd.read_csv(
    'mbatch/bk_mbatch/data/inputs/'+sys+'_'+cancer+'_'+modality+'_'+src_ref+'.tsv',
        sep = '\t',
        index_col = 0)
cp_transposed_read_test

In [ ]:
ab

In [ ]:
ab.System.value_counts()

#### Attach 2 label sets

In [ ]:
ls # this is mbatch_i-o.ipynb

In [ ]:
ls mbatch/bk_mbatch/data/mbatch/

In [ ]:
cd = pd.read_csv(
    'mbatch/bk_mbatch/data/mbatch/correctedResults_src_v1.tsv',
        sep = '\t', index_col = 0)

In [ ]:
cd

In [ ]:
src_df = cd.T[cd.T.index.str.endswith(".src")]
ref_df = cd.T[cd.T.index.str.endswith(".ref")]

In [ ]:
src_df

In [ ]:
ref_df

In [ ]:
ab_src = ab[ab.System == 'cell_line']
ab_ref = ab[ab.System == 'CPTAC']

In [ ]:
ab_src

In [ ]:
ab_ref

In [ ]:
src_df.insert(0, 'Cancer_type', list(ab_src.Cancer_type))
ref_df.insert(0, 'Cancer_type', list(ab_ref.Cancer_type))

In [ ]:
src_df.insert(0, 'System', 'cell_line')
ref_df.insert(0, 'System', 'CPTAC')

In [ ]:
mb_sys = pd.concat([src_df, ref_df], axis = 0)

In [ ]:
mb_sys

#### Run random forest from front-end on mb_sys

In [ ]:
systems = 'cell-line+CPTAC'

In [ ]:
cancer, modality, systems

In [ ]:
mb_sys.to_csv(
    '../results/mbatch/files/'+cancer+'_'+modality+'_'+systems+'.tsv',
    sep = '\t')

In [ ]:
obj_n = pd.read_csv(
    '../results/mbatch/files/'+cancer+'_'+modality+'_'+systems+'.tsv',
    sep = '\t', index_col = 0)

In [ ]:
del(mb_sys)

In [ ]:
import pandas as pd
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import statistics

In [ ]:
obj_n

##### Random forest, for delete

In [ ]:
X = obj_n.iloc[:, 2:]
y_sys = obj_n.System
y_c_typ = obj_n.Cancer_type

sys_scores = []
c_typ_scores = []

for i in list(range(0,9)):
    X_train, X_test, y_train, y_test = train_test_split(X, y_sys)
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average = 'weighted')
    sys_scores.append(score)

    y_train = y_c_typ[y_c_typ.index.isin(y_train.index)]
    y_test = y_c_typ[y_c_typ.index.isin(y_test.index)]
    
    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average = 'weighted')
    c_typ_scores.append(score)

In [ ]:
cancers+'_'+modality+'_'+systems

In [ ]:
file_n = cancers+'_'+modality+'_'+systems

In [ ]:
sys_mean = statistics.mean(sys_scores)
c_typ_mean = statistics.mean(c_typ_scores)

sys_err = statistics.stdev(sys_scores)
c_typ_err = statistics.stdev(c_typ_scores)

fig, ax = plt.subplots()

# Create bars
bar1 = ax.bar('sys', sys_mean, yerr=sys_err, capsize = 7, label='System')
bar2 = ax.bar('c typ', c_typ_mean, yerr=c_typ_err, capsize = 7, label='Cancer type')

plt.title(file_n) # automate later
plt.xlabel('Cancer Model System')
plt.ylabel('F1 score')

# Legend
ax.legend()
plt.savefig('../results/mbatch/figures/'+file_n+'rf_barplot.png')